<a href="https://www.kaggle.com/code/sofiamatias/learning-equality-challenge-semanticsearch?scriptVersionId=121674724" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Imports

In [1]:
import sys, os
sys.path.append("../input/sentence-transformer-package/sentence-transformers-2.2.2/sentence-transformers-2.2.2") 
import sentence_transformers
from sentence_transformers import SentenceTransformer, CrossEncoder, util

In [2]:
import numpy as np
import pandas as pd
import string
import torch

# Loading dataframes

In [3]:
challenge_files_path = '/kaggle/input/learning-equality-curriculum-recommendations'
private_files_path = '/kaggle/input/learningequalityfiles'
model_files_path = '/kaggle/input/sentence-transformer-package'

print (f"\nLoading dataframes...")

for dirname, _, filenames in os.walk(challenge_files_path):
    for filename in filenames:
        filepath = os.path.join(dirname, filename)
        print (f"\nLoading dataframe from {filepath}...")
        df = pd.read_csv (filepath)
        if 'topics' in filepath:
            topics_df = df.fillna({"title": "", "description": ""})
            display(topics_df)
        elif 'sample_submission' in filepath:
            print (f"\nLoading 'sample' dataframe...")
            sample_df = df
            display(sample_df)
        elif 'correlations' in filepath:
            correlations_df = df.fillna({"title": "", "description": ""})
            display(correlations_df)
            print (f"\nCreating exploded correlations 'corr' dataframe")
            corr_df = correlations_df.copy()
            corr_df['content_ids'] = corr_df.content_ids.str.split(' ')
            corr_df = corr_df.explode('content_ids')
            display (corr_df)
        elif 'content' in filepath:
            contents_df = df.fillna({"title": "", "description": "", "text": ""})
            display(contents_df)
print (f"\nDataframes loaded.")


Loading dataframes...

Loading dataframe from /kaggle/input/learning-equality-curriculum-recommendations/sample_submission.csv...

Loading 'sample' dataframe...


,topic_id,content_ids
0,t_00004da3a1b2,c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c...
1,t_00068291e9a4,c_639ea2ef9c95 c_89ce9367be10 c_ac1672cdcd2c c...
2,t_00069b63a70a,c_11a1dc0bfb99
3,t_0006d41a73a8,c_0c6473c3480d c_1c57a1316568 c_5e375cf14c47 c...
4,t_4054df11a74e,c_3695c5dc1df6 c_f2d184a98231



Loading dataframe from /kaggle/input/learning-equality-curriculum-recommendations/topics.csv...


,id,title,description,channel,category,level,language,parent,has_content
0,t_00004da3a1b2,Откриването на резисторите,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True
1,t_000095e03056,Unit 3.3 Enlargements and Similarities,,b3f329,aligned,2,en,t_aa32fb6252dc,False
2,t_00068291e9a4,Entradas e saídas de uma função,Entenda um pouco mais sobre funções.,8e286a,source,4,pt,t_d14b6c2a2b70,True
3,t_00069b63a70a,Transcripts,,6e3ba4,source,3,en,t_4054df11a74e,True
4,t_0006d41a73a8,Графики на експоненциални функции (Алгебра 2 н...,Научи повече за графиките на сложните показате...,000cf7,source,4,bg,t_e2452e21d252,True
...,...,...,...,...,...,...,...,...,...
76967,t_fffb0bf2801d,4.3 Graph of functions,,e77b55,aligned,4,en,t_676e6a1a4dc7,False
76968,t_fffbe1d5d43c,Inscribed shapes problem solving,Use properties of inscribed angles to prove pr...,0c929f,source,4,sw,t_50145b9bab3f,True
76969,t_fffe14f1be1e,Lección 7,,6e90a7,aligned,6,es,t_d448c707984d,True
76970,t_fffe811a6da9,تحديد العلاقة بين الإحداثيّات القطبية والإحداث...,5b9e5ca86571f90499ea987f,9fd860,source,2,ar,t_5b4f3ba4eb7d,True



Loading dataframe from /kaggle/input/learning-equality-curriculum-recommendations/correlations.csv...


...


Creating exploded correlations 'corr' dataframe


,topic_id,content_ids
0,t_00004da3a1b2,c_1108dd0c7a5d
0,t_00004da3a1b2,c_376c5a8eb028
0,t_00004da3a1b2,c_5bc0e1e2cba0
0,t_00004da3a1b2,c_76231f9d0b5e
1,t_00068291e9a4,c_639ea2ef9c95
...,...,...
61513,t_fff9e5407d13,c_d64037a72376
61514,t_fffbe1d5d43c,c_46f852a49c08
61514,t_fffbe1d5d43c,c_6659207b25d5
61515,t_fffe14f1be1e,c_cece166bad6a



Loading dataframe from /kaggle/input/learning-equality-curriculum-recommendations/content.csv...


,id,title,description,kind,text,language,copyright_holder,license
0,c_00002381196d,"Sumar números de varios dígitos: 48,029+233,930","Suma 48,029+233,930 mediante el algoritmo está...",video,,es,NaN,NaN
1,c_000087304a9e,Trovare i fattori di un numero,Sal trova i fattori di 120.\n\n,video,,it,NaN,NaN
2,c_0000ad142ddb,Sumar curvas de demanda,Cómo añadir curvas de demanda\n\n,video,,es,NaN,NaN
3,c_0000c03adc8d,Nado de aproximação,Neste vídeo você vai aprender o nado de aproxi...,document,\nNado de aproximação\nSaber nadar nas ondas ...,pt,Sikana Education,CC BY-NC-ND
4,c_00016694ea2a,geometry-m3-topic-a-overview.pdf,geometry-m3-topic-a-overview.pdf,document,Estándares Comunes del Estado de Nueva York\n\...,es,Engage NY,CC BY-NC-SA
...,...,...,...,...,...,...,...,...
154042,c_fffcbdd4de8b,2. 12: Diffusion,,html5,What will eventually happen to these dyes?\n\n...,en,CSU and Merlot,CC BY-NC-SA
154043,c_fffe15a2d069,Sommare facendo gruppi da 10,Sal somma 5+68 spezzando il 5 in un 2 e un 3.\n\n,video,,it,NaN,NaN
154044,c_fffed7b0d13a,Introdução à subtração,Sal fala sobre o que significa subtrair. Os ex...,video,,pt,NaN,NaN
154045,c_ffff04ba7ac7,SA of a Cone,,video,,en,NaN,NaN



Dataframes loaded.


# Choosing sample data

In [4]:
print ("\nDefining sampled dataset...")
use_submission_sample = False
samples = 100
if ~sample_df.empty and use_submission_sample:
    corr_df = corr_df[corr_df.topic_id.isin(sample_df.topic_id)]
    correlations_df = correlations_df[correlations_df.topic_id.isin(sample_df.topic_id)]
    topics_df = topics_df[topics_df.id.isin(sample_df.topic_id)]
    samples = 5
    print (f"\nFiltered 'topics' to {len(topics_df)} samples and 'contents' to {len(contents_df)} samples")
else:
    topics_df = topics_df[topics_df.has_content == True].sample(n=samples)
    corr_df = corr_df[corr_df.topic_id.isin(topics_df.id)]
    correlations_df = correlations_df[correlations_df.topic_id.isin(topics_df.id)]

display (topics_df)
display (contents_df)
display (correlations_df)


Defining sampled dataset...


,id,title,description,channel,category,level,language,parent,has_content
39487,t_83e4b8d1f351,Real and Ideal Gases,,fef095,source,4,en,t_1c15c4a4eaa6,True
11498,t_26a03d40d0e7,Fish,,fef095,source,4,en,t_319bfb140b11,True
67167,t_dfbeaae7e898,Mantener el equilibrio y girar,,1cc2ee,source,3,es,t_03224ab805cf,True
61057,t_cc0c3b348edf,Quantum mechanical model of atom,,2ee29d,aligned,4,en,t_5796a8466562,True
29189,t_61a597df8c95,మన చెవులు ఏ విధంగా పనిచేస్తాయి?,source_id=LES1542,217993,source,5,te,t_4d527ddcaba4,True
...,...,...,...,...,...,...,...,...,...
45492,t_985fc0dfb082,Les angles formés par deux droites parallèles ...,,c152d6,source,4,fr,t_b799e1436b74,True
26481,t_58cac254cc1a,Solving quadratics using the quadratic formula,Now that we know how to solve quadratics using...,0c929f,source,4,sw,t_fb617d0d7d6c,True
25954,t_570fd29f0870,Construir sucesiones geométricas,Se aprende a construir una sucesión geométrica...,36a98b,source,4,es,t_53d37d45d71e,True
69893,t_e8a8e04ad32f,The Pythagorean Theorem: Geometry’s Most Elega...,This lesson teaches students about the history...,6e3ba4,source,3,en,t_75474744c3bf,True


,id,title,description,kind,text,language,copyright_holder,license
0,c_00002381196d,"Sumar números de varios dígitos: 48,029+233,930","Suma 48,029+233,930 mediante el algoritmo está...",video,,es,NaN,NaN
1,c_000087304a9e,Trovare i fattori di un numero,Sal trova i fattori di 120.\n\n,video,,it,NaN,NaN
2,c_0000ad142ddb,Sumar curvas de demanda,Cómo añadir curvas de demanda\n\n,video,,es,NaN,NaN
3,c_0000c03adc8d,Nado de aproximação,Neste vídeo você vai aprender o nado de aproxi...,document,\nNado de aproximação\nSaber nadar nas ondas ...,pt,Sikana Education,CC BY-NC-ND
4,c_00016694ea2a,geometry-m3-topic-a-overview.pdf,geometry-m3-topic-a-overview.pdf,document,Estándares Comunes del Estado de Nueva York\n\...,es,Engage NY,CC BY-NC-SA
...,...,...,...,...,...,...,...,...
154042,c_fffcbdd4de8b,2. 12: Diffusion,,html5,What will eventually happen to these dyes?\n\n...,en,CSU and Merlot,CC BY-NC-SA
154043,c_fffe15a2d069,Sommare facendo gruppi da 10,Sal somma 5+68 spezzando il 5 in un 2 e un 3.\n\n,video,,it,NaN,NaN
154044,c_fffed7b0d13a,Introdução à subtração,Sal fala sobre o que significa subtrair. Os ex...,video,,pt,NaN,NaN
154045,c_ffff04ba7ac7,SA of a Cone,,video,,en,NaN,NaN


,topic_id,content_ids
1662,t_070136c72382,c_dd739e116435
1814,t_07aa6f66fff1,c_0b0f47b981ae c_441cd0655c03 c_8bc668559da9 c...
1916,t_08209459bab3,c_0cba1468d180 c_5bf34f219262 c_7055be609f46 c...
2567,t_0ac33eb6fc8d,c_0e35a727293f c_5d72ce4a8429 c_68085e5ff02f c...
2795,t_0bbe248a6a3f,c_5b6eacbd3ae4
...,...,...
58337,t_f2df46b8b859,c_999a26ee2c87 c_df444920888a
60118,t_fa595353de71,c_9577f14e1571 c_dd9cfa2e3aab c_f74471a79815 c...
60280,t_faf8c377de07,c_049e5916434a c_1e2058c1313c c_385a9168dec4 c...
60475,t_fbcd034474a4,c_05d32f97e21e c_40579c306c94 c_89d60a25d3f7 c...


# Cleaning data

* Remove ponctuation and special chars from text fields
* Delete columns 'copyright_holder' and 'license' from 'contents'
* Filter 'topics' by 'has_content' = True
* Group 'topics' and 'contents' by language
* Change 'level' column from numbers to text

In [5]:
def clean_text(text_col):
    """
    Clean ponctuation and special chars from a dataframe column
    """
    punctuations = string.punctuation
    text_col = text_col.str.replace('\W', ' ', regex=True)
    for punct in string.punctuation:
        text_col = text_col.str.replace(punct, ' ', regex=True)
    return text_col.str.lower()

In [6]:
# Cleaning topics
levels = {1: 'Level 1', 2: 'Level 2', 3: 'Level 3', 4: 'Level 4', 5: 'Level 5', 6: 'Level 6', 7: 'Level 7', 
          8: 'Level 8', 9: 'Level 9', 10: 'Level 10', 0: 'Level 0'}
topics_cols = ['title', 'description']

print (f"\nCreating and cleaning topic features...")
topic_features = topics_df.copy()
topic_features = topic_features.replace ({'level': levels})
for col in topics_cols:
    topic_features[col] = clean_text(topic_features[col])
topic_features.sort_values (by='language', inplace=True)

#topics_features['sentences'] = topics_features[topics_cols].apply(lambda x: '.'.join(x.dropna().astype(str)), axis=1)
#topics_features = topics_features.drop(columns=['parent'] + topics_cols) 
print (f"\nCreated 'topic_features'")
display (topic_features)


Creating and cleaning topic features...

Created 'topic_features'


,id,title,description,channel,category,level,language,parent,has_content
41054,t_892e5157f6ef,مفهوم الاحتكاك وخواصه و بعض التطبيقات الحياتية...,5b9e2c4c6571f9049ab7a82b,9fd860,source,Level 2,ar,t_df43a225fa13,True
18970,t_3fabd57d528d,يمي ز الأعداد الأو لي ة والأعداد المرك بة ويحل...,5a4c84387dd197090857ee17,9fd860,source,Level 3,ar,t_2366c208b0c1,True
26688,t_5971829c0edf,إجراء العملي ات على مقادير جبري ة,5b9e5171b1fcd3049c229ddb,9fd860,source,Level 2,ar,t_700a6c6c9d95,True
2291,t_07aa6f66fff1,السوق الاقتصاد ج 3,التعريف بعلم الاقتصاد توزيع الموارد والثروات م...,a13d2e,source,Level 3,ar,t_51e7c47560bc,True
29776,t_639930e1909b,م ي مون ة في إ ث يوب يا,,7b47c5,source,Level 4,ar,t_5c935eadb393,True
...,...,...,...,...,...,...,...,...,...
29189,t_61a597df8c95,మన చ వ ల ఏ వ ధ గ పన చ స త య,source id les1542,217993,source,Level 5,te,t_4d527ddcaba4,True
20943,t_468b8a1af0be,混合运算的运算法则,在本教程中 我们会学习 运算顺序 来解涉及指数 括号 乘除 和加减的复杂算式,f83dcf,source,Level 4,zh,t_6b971ca5a087,True
24458,t_522c4a48a586,人寿保险,虽然这么想让人有些沮丧 但事实上我们所有人最终都将死去 你是否会担心你所爱的人将面临什么呢 ...,f83dcf,source,Level 4,zh,t_eb4921ed5d8f,True
41174,t_899312649758,轴对称图形,认识轴对称图形,f83dcf,source,Level 4,zh,t_7ff2350f7f86,True


In [7]:
# Cleaning contents
content_cols = ['title']


print (f"\nCreating and cleaning content features...")
content_features = contents_df.copy()
for col in content_cols:
    content_features[col] = clean_text(content_features[col])
#content_features['sentences'] =  content_features[content_cols].apply(lambda x: '.'.join(x.dropna().astype(str)), axis=1)
content_features.sort_values (by='language', inplace=True)
content_features.drop(columns=['copyright_holder', 'license'], inplace=True)
print (f"\nCreated 'content_features'")
display(content_features)


Creating and cleaning content features...

Created 'content_features'


,id,title,description,kind,text,language
133868,c_de43cff8dd60,المفردات والتراكيب,,exercise,"ما المقصود بعبارة: ""هل هناك من خطب""؟\n\n- هل ه...",ar
63589,c_699d4bd5d5bb,المفردات والتراكيب,,exercise,"ما ضدّ ""لا تملّ"" في الجملة الآتية: ""تُعيدُ الأ...",ar
17214,c_1cab88377df5,المفردات والتراكيب,,exercise,"ما مرادف ""يتمّم"" في ما يلي: ""يَقِفُ أَحْيانًا ...",ar
145550,c_f1b8123b975e,الس ف ر,"يركب ""رامي"" الطّائرة للمرّة الأولى فيحقّق أمله...",html5,"السَّفَرُ\n\nبِحِرْصٍ شَديدٍ، حَزَمَ ""رامي"" أَ...",ar
17202,c_1ca6079deb47,يعي ن معكوس مصفوفة مرب عة من الرتبة الثالثة با...,5ad46b8a6b9064043d8b4158,exercise,![](${☣ CONTENTSTORAGE}/40cf62978d6581e03a5085...,ar
...,...,...,...,...,...,...
15012,c_18fc5ad44e5d,构建比较级和最高级,学习如何改装比较级和最高级修饰语\n\n,video,,zh
128758,c_d5e56175090a,主谓一致简介,主谓一致指匹配一个句子的主语和动词！这里介绍它是怎么一回事.\n\n,video,,zh
32558,c_35f7982b250f,双臂和胯部热身运动,在本视频中，您将学会三个热身练习，锻炼您上半身的肌肉，尤其是双臂和胯部的肌肉。,document,\n双臂和胯部热身运动\n双臂和胯部热身运动\n在本视频中，您将学会三个热身练习，锻炼您上...,zh
15053,c_19148c5d7ab3,出血,微博：http://www.weibo.com/sikana\r\n脸书：https://w...,document,\n出血\n本视频由红十字会和Sikana共同出品\n急救\n出血\n在这个视频您将学习怎...,zh


# Scoring: F2 score 

In [8]:
def calculate_F2score(pred_df, act_df):
    
    """
    Using predictions_df and actual_df as exploded correlation columns to calculate F1 score.
    Results show correct predicts, recall, precision and F2 score.
    Results also return the list of correct predicts, correct_df_
    """
    print ('\nCalculating scores...')
    if pred_df.empty or act_df.empty:
        print ('\nOne or both dataframes are empty. Abort F2score calculation.')
        return None
    prediction_df=pred_df.copy()
    actual_df = act_df.copy()
    prediction_df.columns=['topic_id', 'content_ids_pred']
    actual_df.columns=['topic_id', 'content_ids_actual']
    df = pd.merge(prediction_df, actual_df, how='inner', on='topic_id')
    if df.empty:
        print ('\nNo matches between predictions and correlations. Abort F2score calculation.')
        return None
    df['tp'] = df[['content_ids_pred', 'content_ids_actual']].apply (lambda x: len(set(x['content_ids_actual'].split()).intersection(set(x['content_ids_pred'].split()))), axis=1)
    df['fp'] = df[['content_ids_pred', 'content_ids_actual']].apply (lambda x: len(set(x['content_ids_pred'].split()) - set(x['content_ids_actual'].split())), axis=1)
    df['fn'] = df[['content_ids_pred', 'content_ids_actual']].apply (lambda x: len(set(x['content_ids_actual'].split()) - set(x['content_ids_pred'].split())), axis=1)
    df['precision'] = df['tp'] / (df['tp'] + df['fp'])
    df['recall'] = df['tp'] / (df['tp'] + df['fn'])
    df['f2'] = df['tp'] / (df['tp'] + 0.2 * df['fp'] + 0.8 * df['fn']) 
    print ('\nF2 score calculation finished.')

    return df

# Getting matches: sentence transformer with retrain-rerank

In [9]:
def search(query, topic_embedding, corpus_embeddings, content_sentences, content_ids, cross_encoder, top_k):

    # passages = content_sentences

    ##### Semantic Search #####
    # find potentially relevant passages
    hits = util.semantic_search(topic_embedding, corpus_embeddings, top_k=100)
    hits = hits[0]  # Get the hits for the first query

    ##### Re-Ranking #####
    # Now, score all retrieved passages with the cross_encoder
    cross_inp = [[query, content_sentences[hit['corpus_id']]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)

    # Sort results by the cross-encoder scores
    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]

    # Output of top-30 hits from re-ranker
    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    results = {}
    for hit in hits[0:top_k]:
        results[content_ids.iloc[hit['corpus_id']]] = content_sentences[hit['corpus_id']]
    return results

In [10]:
from tqdm import tqdm

gpu_on = True

if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")
    gpu_on = False
    
languages = topic_features.language.unique()
print (languages)
preds = {}
matches = {}
biencoder = "/kaggle/input/learning-equality-st-train-sm/ST-all-MiniLM-L6-v2-trained"
crossencoder = '/kaggle/input/msmarcominilml6v2/ms-marco-MiniLM-L-6-v2'

print (f"\nGetting matches using bi-encoder {biencoder} and cross encoder {crossencoder}...")

#Use the Bi-Encoder to encode all contents, so that we can use it with semantic search
bi_encoder = SentenceTransformer(biencoder)
bi_encoder.max_seq_length = 256    #Truncate long passages to 256 tokens (256 is ideal value)
top_k = 5                         #Number of passages we want to retrieve with the bi-encoder (10 is ideal value)

#Use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder(crossencoder)

for lang in languages:
    print ('\nWorking on topics for language ', lang)
    content_sentences = content_features[content_features.language == lang]
    topic_sentences = topic_features[topic_features.language == lang]

    if len(content_sentences) == 0:
        print ('\nNo contents for this language.')
        continue
    if len(topic_sentences) == 0:
        print ('\nNo topics for this language.')
        continue

    
    print ("\nCalculating 'content' embeddings...")

    # encode all contents into our vector space. This takes about 5 minutes (depends on your GPU speed)
    corpus_embeddings = bi_encoder.encode(content_sentences.title.to_list(), convert_to_tensor=True, show_progress_bar=True)
    
    if gpu_on:
        corpus_embeddings = corpus_embeddings.cuda()
        corpus_embeddings = util.normalize_embeddings(corpus_embeddings)
    
    print ("\nCalculating 'topic' embeddings...")
    
    # Encode the topics using the bi-encoder
    topic_embeddings = bi_encoder.encode(topic_sentences.title.to_list(), convert_to_tensor=True, show_progress_bar=False)
    
    if gpu_on:
        topic_embeddings = topic_embeddings.cuda()
        topic_embeddings = util.normalize_embeddings(topic_embeddings)
    
    print ("\nRunning matches...")
    
    for i, (topic_embed, query) in enumerate(tqdm(zip (topic_embeddings, topic_sentences.title.to_list()), total=len(topic_sentences))):
        results = search(query,
                         topic_embed,
                         corpus_embeddings, 
                         content_sentences.title.to_list(), 
                         content_sentences.id,
                         cross_encoder,
                         top_k)
        matches[query] = results.values()
        preds[topic_sentences.iloc[i].id] = results.keys()
print ('\nEnd of calculating matches.')

['ar' 'bg' 'bn' 'en' 'es' 'fil' 'fr' 'hi' 'mr' 'or' 'pt' 'sw' 'te' 'zh']

Getting matches using bi-encoder /kaggle/input/learning-equality-st-train-sm/ST-all-MiniLM-L6-v2-trained and cross encoder /kaggle/input/msmarcominilml6v2/ms-marco-MiniLM-L-6-v2...

Working on topics for language  ar

Calculating 'content' embeddings...


Batches:   0%|          | 0/232 [00:00<?, ?it/s]


Calculating 'topic' embeddings...

Running matches...


  0%|          | 0/9 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

 75%|███████▌  | 3/4 [00:01<00:00,  2.27it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:02<00:00,  1.96it/s]


End of calculating matches.


In [11]:
# predictions in exploded format
preds_df = pd.DataFrame(zip (list(preds.keys()), [list(vals) for vals in preds.values()]), columns=['id','content_ids']).explode('content_ids')

# predictions in submissions format
df_preds_aux = pd.DataFrame(zip (list(preds.keys()), (' '.join(list(preds[key])) for key in preds.keys())), columns=['id','content_ids'])

predicts_submission = pd.DataFrame(topics_df.id).merge (df_preds_aux, how ='left', on = 'id')
predicts_submission.rename(columns={'id':'topic_id'}, inplace=True)
predicts_submission.fillna(' ', inplace=True)

predicts_submission
#predicts_submission.content_ids.apply (lambda x: len(x.split()))

,topic_id,content_ids
0,t_83e4b8d1f351,c_201eef50c13a c_89133219d38f c_898a64863f8f c...
1,t_26a03d40d0e7,c_52b90b3c9f0c c_f6bbf17613fe c_5d599261a245 c...
2,t_dfbeaae7e898,c_9f5c11bec6b5 c_b2a20d66445c c_f901a17ba7ea c...
3,t_cc0c3b348edf,c_e7e82fa33a2c c_fc7fcb2413b5 c_7988165fd565 c...
4,t_61a597df8c95,c_ce66fdc0f6b9 c_5cdc7929d08d c_488bb38ee075 c...
...,...,...
95,t_985fc0dfb082,c_e0335a8e5520 c_e00f04595934 c_792336e8ea85 c...
96,t_58cac254cc1a,c_73dbdd071fdc c_d703cff79c12 c_eb2eaa45cc59 c...
97,t_570fd29f0870,c_a9e96210069b c_7239db7a008f c_66fc1ba3e23c c...
98,t_e8a8e04ad32f,c_4745e552cb18 c_863b6696e0e8 c_95a794f8d037 c...


# Model evaluation

In [12]:
scoring = True
if scoring:
    score = calculate_F2score(predicts_submission.sort_values('topic_id'), correlations_df)
    if score is not None:
        print(score)
        print ('F2 mean score:', score.f2.mean())
        print ('Correct predictions:', score.tp.sum())
        print ('Topics to match:', len(topics_df))
        print ('False positives:', score.fp.sum())
        print ('False negatives:', score.fn.sum())
        print ('Contents to match:', len(corr_df.merge (topics_df.id, how = 'inner', left_on='topic_id', right_on='id')))


Calculating scores...

F2 score calculation finished.
          topic_id                                   content_ids_pred  \
0   t_070136c72382  c_5dc764098035 c_60505f3d9436 c_fcebc5f7e82f c...   
1   t_07aa6f66fff1  c_d7a814206f68 c_0b0f47b981ae c_981912346ebb c...   
2   t_08209459bab3  c_0cba1468d180 c_92cc3afba44d c_410652abfba3 c...   
3   t_0ac33eb6fc8d  c_9c4feb7dfe34 c_81a0ceacfe9a c_e1cb8c8682f8 c...   
4   t_0bbe248a6a3f  c_8e0e69274e81 c_4c700cd34feb c_5038d4fef2df c...   
..             ...                                                ...   
95  t_f2df46b8b859  c_df444920888a c_3e47660eb113 c_6f5ce7a5d12b c...   
96  t_fa595353de71  c_f302833218fb c_c2e3ae51ebd0 c_43c85d03390f c...   
97  t_faf8c377de07  c_6292a8db943b c_9cc3d11a8a42 c_ada1a4df1dac c...   
98  t_fbcd034474a4  c_e2d98845c44f c_05d32f97e21e c_1e53f4a7c6f2 c...   
99  t_fe378591d4c9  c_2a28b527f0a6 c_88cb6170150d c_49586d0c2669 c...   

                                   content_ids_actual  tp  fp  fn  p

# Submissions

In [13]:
#predicts_submission.to_csv('submission.csv', index=False)

# Cleaning wrong matches with LGBM

In [14]:
clean_wrong_matches = False

## Calculating features 'X_test' for predictions

In [15]:
def categorize_features (X: pd.DataFrame):
    obj_feat = list(X.loc[:, X.dtypes == 'object'].columns.values)
    for feature in obj_feat:
        X[feature] = pd.Series(X[feature], dtype="category")
    return X
    
def get_y_class (y):
    return (y > 0.5).astype("bool")

In [16]:
if clean_wrong_matches:
    X_test = preds_df.merge (topic_features, how='inner', on='id')
    X_test.drop (columns = ['language', 'has_content', 'description'], inplace=True)
    X_test.rename (columns = {'id': 'topic_id', 'title':'topic_title'}, inplace=True)
    X_test = X_test.merge (content_features, how='inner', left_on='content_ids', right_on='id')
    X_test.drop (columns = ['title', 'description', 'text', 'id'], inplace=True)
    X_test = categorize_features (X_test)
    display (X_test, X_test.dtypes)

In [17]:
#corr_df
#preds_df = preds_df.merge (corr_df, how='left', on = 'content_ids')
#preds_df['match'] = (preds_df.id == preds_df.topic_id)
#preds_df

# Get model (load existing model or train model)

In [18]:
import lightgbm as lgb

if os.path.exists(private_files_path) and clean_wrong_matches:
    for dirname, _, filenames in os.walk(private_files_path):
        for filename in filenames:
            filepath = os.path.join(dirname, filename)
    model = lgb.Booster(model_file=filepath)
    print ('Loaded model from :', filepath)
elif clean_wrong_matches:
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import precision_score
    import lightgbm as lgb
    import optuna
    import datetime
        
    objective = 'binary'
    boosting_type = 'dart'

    def objective_lgbm(trial):
    
        param = {
            'boosting_type': boosting_type,
            'objective': objective,
            'is_unbalance': 'true',
            'metric': 'precision_score',
            'pos_bagging_fraction': trial.suggest_float('pos_bagging_fraction',0.1,1),
            'neg_bagging_fraction': trial.suggest_float('neg_bagging_fraction',0.1,1),
            'num_leaves': trial.suggest_int('num_leaves', 10,60),
            'max_depth': trial.suggest_int('max_depth', 10,60),
            'min_split_gain': trial.suggest_float('min_split_gain',0.1,1),
            'colsample_bytree': trial.suggest_float('colsample_bytree',0.1,1),
            'reg_alpha' : trial.suggest_float('reg_alpha',0.1,10),
            'reg_lambda': trial.suggest_float('reg_lambda',0.1,10),
            'n_estimators': trial.suggest_int('n_estimators', 150,350),
            'learning_rate': trial.suggest_float('learning_rate', 0.01,0.1),
            'verbosity': -1
        }
        num_boost_rounds = trial.suggest_int('num_boost_rounds', 50,400)
        model = lgb.train(param, train_data, num_boost_rounds)
        preds = model.predict(X_val)
        pred_labels = np.rint(preds)
        score = round(precision_score(y_val, pred_labels),4)
        return score
    
    # building a dataframe 'mix_matches' with 50% true topic-content matches and 50% false topic-content matches
    df_aux = preds_df.merge (corr_df, how='left', on='content_ids')
    true_matches = df_aux[df_aux['id'] == df_aux['topic_id']]
    true_matches_val = len(df_aux[df_aux['id'] == df_aux['topic_id']])
    false_matches = df_aux[df_aux['id'] != df_aux['topic_id']].sample(n=samples)
    mix_matches = pd.concat([true_matches, false_matches])
    mix_matches['y'] = (mix_matches.id == mix_matches.topic_id)
    mix_matches.drop (columns='topic_id', inplace=True)

    # building X features and y target 
    y = mix_matches['y']
    X = mix_matches.merge (topic_features, how='inner', on='id')
    #X = X.merge (content_features, how='inner', left_on='content_ids', right_on='id')
    X.drop (columns = ['language', 'has_content', 'description', 'y'], inplace=True)
    X.rename (columns = {'id': 'topic_id', 'title': 'topic_title'}, inplace=True)
    X = X.merge (content_features, how='inner', left_on='content_ids', right_on='id')
    X.drop (columns = ['title', 'description', 'text', 'id'], inplace=True)

    # preparing X and y for model: changing dtype to "category" and creating train/val sets
    X = categorize_features (X)

    print(f"Training features of shape {X.shape}")
    display (X)
    print(f"Training labels of shape {y.shape}")
    display (y)
    
    X_train, X_val, y_train, y_val = train_test_split (X, y, test_size = 0.3, random_state=42)

    train_data = lgb.Dataset(X_train, label = y_train)

    # get best hyperparameters
    study_lgbm = optuna.create_study(direction = 'maximize',study_name = "LGBM")
    study_lgbm.optimize(objective_lgbm, n_trials=50)

    trial_lgbm = study_lgbm.best_trial
    print("Model Accuracy --> ",trial_lgbm.value)
    print("Model's Best parameters --> ",trial_lgbm.params)

    # fit model and get score
    num_boost_rounds = trial_lgbm.params['num_boost_rounds']
    del trial_lgbm.params['num_boost_rounds']
    trial_lgbm.params['boosting_type'] = boosting_type
    trial_lgbm.params['objective'] = objective
    trial_lgbm.params['is_unbalance'] = True
    trial_lgbm.params['verbosity'] = -1
    print("Using parameters --> ",trial_lgbm.params)
    model = lgb.train(trial_lgbm.params, train_data, num_boost_rounds)
    pred_model = model.predict(X_val)
    pred_model = get_y_class (pred_model)
    score = precision_score(y_val, pred_model)
    print('\nLightGBM Model accuracy score: {0:0.4f}'.format(score))
    from sklearn.metrics import confusion_matrix
    print('\nConfusion Matrix : \n' + str(confusion_matrix(y_val,pred_model)))
    
    # save model
    date = datetime.datetime.now().strftime('%d%m%y-%H%M')
    model_filename = f"lgb-classifier-{date}.txt"
    model.save_model (model_filename)
    print ('\nSaved model as ', model_filename)

# New Predictions

In [19]:
if clean_wrong_matches:
    # get predictions
    preds_lgbm = model.predict(X_test)
    preds_lgbm = get_y_class (preds_lgbm)
    print ('\nPredictions: ', preds_lgbm[-50:])
    print (pd.DataFrame(preds_lgbm).value_counts())
    #put predictions in final format
    df_preds_aux = preds_df[preds_lgbm]
    print (df_preds_aux.groupby('id').count())
    df_preds_aux = df_preds_aux.groupby(['id']).apply (lambda x : ' '.join (x.iloc[:, 1])).reset_index()
    df_preds_aux.rename (columns = {0: 'content_ids'}, inplace=True)
    print (df_preds_aux)
    predicts_submission = pd.DataFrame(topics_df.id).merge (df_preds_aux, how ='left', on = 'id')
    predicts_submission.rename(columns={'id':'topic_id'}, inplace=True)
    predicts_submission.fillna(' ', inplace=True)

    display(predicts_submission)
    
    # submissions
    predicts_submission.to_csv('submission.csv', index=False)

In [20]:
predicts_submission.content_ids.apply (lambda x: len(x.split()))

0     5
1     5
2     5
3     5
4     5
     ..
95    5
96    5
97    5
98    5
99    5
Name: content_ids, Length: 100, dtype: int64

# Final Score

In [21]:
if clean_wrong_matches and scoring:
    score = calculate_F2score(predicts_submission.sort_values('topic_id'), correlations_df)
    if score is not None:
        display(score)
        print ('F2 mean score:', score.f2.mean())
        print ('Correct predictions:', score.tp.sum())
        print ('Topics to match:', len(topics_df))
        print ('False positives:', score.fp.sum())
        print ('False negatives:', score.fn.sum())
        print ('Contents to match:', len(corr_df.merge (topics_df.id, how = 'inner', left_on='topic_id', right_on='id')))

In [22]:
predicts_submission.to_csv('submission.csv', index=False)